To use this example notebook, an API key from OpenAI will be need. It is necessary to use OpenAI's embedding and GPT models.  

### Installs & imports

In [ ]:
%%capture
!pip install "openai==1.17"

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import zscore

from openai import OpenAI

from google.colab import files

### Constants

Available variables:
*   OPEN_AI_KEY: Your private API key from OpenAI platform. Remember not to share it publicly.
*   LLM_MODEL: One of possible GPT models for generating the sql queries.
*   TEMPERATURE: Variance in the possible model's responses. Lower number is preffered.
*   Z_SCORE_RANGE: The higher the value, the more additional tables (from the database) may be provided to the LLM. *This score is used to rank the similarity between the tables and given natural language query.*



In [ ]:
OPEN_AI_KEY = '' # @param {type:"string"}
LLM_MODEL = "gpt-4-turbo-preview" # @param ["gpt-4-turbo-preview", "gpt-4", "gpt-3.5-turbo"] {type:"raw"}
TEMPERATURE = 0.1 # @param {type:"slider", min:0, max:0.4, step:0.02}
 ### Low number preffered for less mistakes
FREQUENCY_PENALTY = -1
Z_SCORE_RANGE = 0.7 # @param {type:"slider", min:0.2, max:1.0, step:0.02}
### The higher the above number, the more additional tables may be provided to the LLM
LLM_CONTEXT = "You are writing SQL queries only. For every user question, answer only with an SQL query which does what the user wants, on a provided table."
### Potential improvement: Optimize the length of the context to save money on tokens

### Definitions and generator class

In [ ]:
cosine_similarity_numpy = lambda vec1, vec2: np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))
### Alternatively I could use scipy, or scikit-learn, but this pure numpy approach is very fast and flexible if needed

In [ ]:
### OpenAI

client = OpenAI(
    api_key = OPEN_AI_KEY
)

def get_openai_embedding(text, model="text-embedding-3-small"):
  return client.embeddings.create(input = [text], model=model).data[0].embedding

def get_response(user_query, table_queries, context=None, frequency_penalty=None, temperature=None):
  if context == None:
    context = LLM_CONTEXT
  if frequency_penalty == None:
    frequency_penalty = FREQUENCY_PENALTY
  if temperature == None:
    temperature = TEMPERATURE
  response = client.chat.completions.create(
    model = LLM_MODEL,
    temperature = temperature,
    frequency_penalty = frequency_penalty,
    messages=[
      {"role": "system", "content": context},
      {"role": "user", "content": f"User question: {user_query}. Tables: {table_queries}  Write an SQL query that satisfies the user's question based on the provided tables"}
    ]
  )
  return response

def parse_response(response, remove_markings=True):
  answer = response.choices[0].message.content
  return answer.removeprefix('```sql').removesuffix('```') if remove_markings else answer

def get_description_response(query):
  response = client.chat.completions.create(
    model = LLM_MODEL,
    temperature = 0.5,
    messages=[
      {"role": "system", "content": "You are taking in SQL queries and make short descriptions (without technical details) of what information they store."},
      {"role": "user", "content": query}
    ]
  )
  return response


In [ ]:
class QueryGenerator():

  def __init__(self, df):
    self.df = df

  def create_embeddings(self, descriptions=True):
    def insert_vectors(row):
      if descriptions:
        description = parse_response(get_description_response(row['query']))
        return get_openai_embedding(description)
      else:
        return get_openai_embedding(row['query'])
    self.df['vector'] = self.df.apply(insert_vectors, axis=1)

  def check_similarities(self, nl_query:str):
    nl_vector = get_openai_embedding(nl_query)
    cos_similarities = []
    for idx, row in self.df.iterrows():
      cos_similarities.append(cosine_similarity_numpy(row['vector'], nl_vector))
    return cos_similarities

  def generate_query(self, nl_query:str, zscore_range=None):
    if zscore_range == None:
      zscore_range = Z_SCORE_RANGE
    cos_similarities = self.check_similarities(nl_query)
    max_idx = np.argmax(np.array(cos_similarities))
    z_scores = zscore(cos_similarities).tolist()
    z_threshold = z_scores[max_idx] - zscore_range
    relevant_idx = [z_scores.index(score) for score in z_scores if score > z_threshold]
    print("The chosen tables:", relevant_idx) ### Left for testing purposes
    table_queries = self.df.iloc[relevant_idx, 1].tolist()
    response = get_response(nl_query, table_queries)
    return parse_response(response)


### Testing

In [ ]:
### Loading Pandas dataframe from CSV.
print("Upload CSV ['id','query'] with SQL queries:")
uploaded = files.upload()
query_f = list(uploaded.keys())[0]
query_df = pd.read_csv(query_f)

### You can use the csv from the package, or upload your own with the similar format.

Upload CSV ['id','query'] with SQL queries:


Saving dummy_sql_tables - Arkusz1.csv to dummy_sql_tables - Arkusz1 (1).csv


In [ ]:
qg = QueryGenerator(query_df)
qg.create_embeddings()

In [ ]:
# @markdown Select from list or type Your own.
NATURAL_LANGUAGE_QUERY = "What percentage of our users are located in Warsaw, Poland?" # @param ["How many transactions did make the user who wrote the most user rewievs?", "Do we have more shipments Italy or France?", "What is the adress of a person who made the last order?", "What percentage of our users are located in Warsaw, Poland?"] {allow-input: true}
qg.generate_query(NATURAL_LANGUAGE_QUERY)

The chosen tables: [9]


"\nSELECT\n    ROUND((COUNT(*) FILTER (WHERE city = 'Warsaw' AND country = 'Poland') * 100.0) / COUNT(*), 2) AS percentage_users_in_warsaw\nFROM\n    testbase.shipping_addresses;\n"